In [7]:

import numpy as np
import random
from numpy.linalg import*
import matplotlib.pyplot as plt
from numpy import r_, exp, cos, sin, pi, zeros, ones, hanning, sqrt, log, floor, reshape, mean
from scipy import signal
from numpy.fft import fft
import math
import time
import scipy.optimize as opt

import scipy.io as sio
import scipy.io.wavfile
import sounddevice as sd
from IPython.display import Audio

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import copy
get_ipython().run_line_magic('matplotlib', 'inline')
import librosa
import librosa.display

import importlib


import functions as lib_HS 
importlib.reload(lib_HS)

import itertools 

import ConvBSS_IVA_NG 
importlib.reload(ConvBSS_IVA_NG)
from ConvBSS_IVA_NG import *


import MIC_Signal_Mod as MIC_Access
importlib.reload(MIC_Access)


<module 'MIC_Signal_Mod' from 'G:\\My Drive\\Personal_Folder\\Python_Codes\\SeeCorner\\MUSE_codes\\MIC_Signal_Mod.py'>

In [3]:
## LibriTTS dataset import
N_test = 11
N_len = 100000
TTS = np.zeros((N_test,N_len))
for i in range(1,N_test+1):

    TTS_i_raw, fs_MIC = librosa.load('./TTS_dataset/TTS_norm_'+str(i)+'.wav', sr=None)
    TTS[i-1,:] = TTS_i_raw[:N_len]


print(np.linalg.norm(TTS,ord=2,axis=1))
Data_perm = np.array([[0,1,2],[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7],[6,7,8],[7,8,9],[8,9,10],[9,10,0],[10,0,2],
                     [0,2,4],[2,4,6],[4,6,8],[10,1,3],[1,3,5],[3,5,7],[5,7,9],[7,9,0],[0,3,6],[3,6,9]])
#Data_numsrc = np.array([[10],[10,1],[10,1,2],[10,1,2,3],[10,1,2,3,4]])
Data_numsrc = np.array([[0],[0,1],[0,1,2],[0,1,2,3],[0,1,2,3,4]])

[10.81605919  9.08186293 11.98752459 10.00808383 11.56235343  7.60182595
  9.91867348 11.150898    9.29773915  9.57081537 10.85136336]


<ipython-input-3-e29d531107a6>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data_numsrc = np.array([[0],[0,1],[0,1,2],[0,1,2,3],[0,1,2,3,4]])


In [4]:
############# MIC parameter
d = 0.05 ## 5 cm 
SNR = 15
############# Room parameter
Range = 3          # in meter
Room_len = 10      # in meter
z = 1000
T_reverb = 0.45    ## in second
nsample = 4096
dim = 2
order = 0
vp = 343
#################################

In [8]:
### IVA BSS
Angle = [170, 70, -30]
N_fft = 512
N_hop = int(0.25*N_fft)
lr = 0.1
maxiter = 1000
tol = 1e-6

M = 2  # number of MIC
K = 3  # number of source
 
SigVec = TTS[Data_numsrc[K-1],:]
X, _ = MIC_Access.MixGen_RIR(SigVec, Angle[:K], 0, fs_MIC, SNR, vp, d, Range, Room_len, z, T_reverb, nsample, dim, order)


y_t, Y, A = ConvBSS_IVA_NG(X, fs_MIC, N_fft, N_hop, None, lr, maxiter, tol)



1 iterations: Objective= -0.03389610769065431 , dObj= 0.08570313593821666
21 iterations: Objective= -1.447722025481689 , dObj= 0.051130326423317385
41 iterations: Objective= -1.9525130938381292 , dObj= 0.007279693269524046
61 iterations: Objective= -1.991684958376543 , dObj= 0.00021085192152092702
81 iterations: Objective= -1.99317948174512 , dObj= 6.271935877810719e-05
101 iterations: Objective= -1.9963147832713057 , dObj= 0.0003077255989847316
121 iterations: Objective= -2.012363461930432 , dObj= 0.0014979808211985635
141 iterations: Objective= -2.057816254960429 , dObj= 0.0021503690994708613
161 iterations: Objective= -2.0773039171324124 , dObj= 0.00041603791357536224
181 iterations: Objective= -2.0822920146743864 , dObj= 0.00013416360596263388
201 iterations: Objective= -2.083668567208772 , dObj= 3.191639537769575e-05
221 iterations: Objective= -2.0840249001229356 , dObj= 1.019926829437523e-05
241 iterations: Objective= -2.0841599908492823 , dObj= 4.584586014377834e-06
261 iteratio

In [ ]:
f = np.linspace(0,fs_MIC/2, num=int(N_fft/2+1))
fmax = 5e3
f_clip_min = 300
f_clip_max = fmax/2.5


    
### Angle estim. and variances ###

f_L = np.array(np.where(f<f_clip_max)).flatten()
r_ji = np.zeros((len(f_L), 2), dtype='float')

for freq in f_L:
    if freq==0:
        continue
    for i in range(2):
        r_ji[freq,i] = (vp/d)*np.angle(A[1,i,freq]/A[0,i,freq])/(2*np.pi*f[freq])    #refernce MIC index = 0
        #r_ji[freq,i] = np.arccos((vp/d)*np.angle(A_f_jica[freq,1,i]/A_f_jica[freq,0,i])/(2*np.pi*f[freq]))

        # r_f : freq x src #
## Cluters

r_feat_0 = lib_HS.rej_outlier(r_ji[np.min(np.where(f>f_clip_min)):,0])
r_feat_1 = lib_HS.rej_outlier(r_ji[np.min(np.where(f>f_clip_min)):,1])
clus_k = np.array([np.average(r_feat_0), np.average(r_feat_1)])
clus_k_std = np.array([np.std(r_feat_0), np.std(r_feat_1)])

clist = ['r','b','g']


for i in range(2):

    plt.plot(f[f_L[np.min(np.where(f>f_clip_min)):]],r_ji[np.min(np.where(f>f_clip_min)):,i], color=clist[i],marker='o',ms=2,ls="None",label=i)
    plt.plot(clus_k[i], color=clist[i],marker='o',ms=10, mec='Cyan')

#plt.title('Truth AoAs[deg] : '+ str(Angle), fontweight='bold')
plt.xlabel("freq[Hz]")
plt.ylabel("in rad")
plt.grid(ls='--')    
plt.legend()
plt.show()

print('Average features = ',clus_k)
print('STDs = ',clus_k_std)

'''
angle_store_1 = []
angle_store_2 = []
for i in range(5,55):
    angle_store_1.append(np.angle(A[1,0,i]/A[0,0,i])/(2*np.pi*f[i]))
    angle_store_2.append(np.angle(A[1,1,i]/A[0,1,i])/(2*np.pi*f[i]))


plt.plot(angle_store_1,'.')
plt.plot(angle_store_2,'.')   ''' 

In [ ]:
plt.imshow(abs(Y[1,:]))

In [ ]:
print(np.var(angle_store_1),np.var(angle_store_2))   ## K = 2

In [ ]:

f

In [ ]:
Audio(y_t[0],rate=fs_MIC)

In [ ]:
Audio(y_t[1],rate=fs_MIC)

In [ ]:
test = np.array([[1,2],[3,4]])

In [ ]:
sum(sum(test))